![dvd_image](dvd_image.jpg)

A DVD rental company needs your help! They want to figure out how many days a customer will rent a DVD for based on some features and has approached you for help. They want you to try out some regression models which will help predict the number of days a customer will rent a DVD for. The company wants a model which yeilds a MSE of 3 or less on a test set. The model you make will help the company become more efficient inventory planning.

The data they provided is in the csv file `rental_info.csv`. It has the following features:
- `"rental_date"`: The date (and time) the customer rents the DVD.
- `"return_date"`: The date (and time) the customer returns the DVD.
- `"amount"`: The amount paid by the customer for renting the DVD.
- `"amount_2"`: The square of `"amount"`.
- `"rental_rate"`: The rate at which the DVD is rented for.
- `"rental_rate_2"`: The square of `"rental_rate"`.
- `"release_year"`: The year the movie being rented was released.
- `"length"`: Lenght of the movie being rented, in minuites.
- `"length_2"`: The square of `"length"`.
- `"replacement_cost"`: The amount it will cost the company to replace the DVD.
- `"special_features"`: Any special features, for example trailers/deleted scenes that the DVD also has.
- `"NC-17"`, `"PG"`, `"PG-13"`, `"R"`: These columns are dummy variables of the rating of the movie. It takes the value 1 if the move is rated as the column name and 0 otherwise. For your convinience, the reference dummy has already been dropped.

In [69]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# Import any additional modules and start coding below

In [70]:
df_rental = pd.read_csv('rental_info.csv')
df_rental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   rental_date       15861 non-null  object 
 1   return_date       15861 non-null  object 
 2   amount            15861 non-null  float64
 3   release_year      15861 non-null  float64
 4   rental_rate       15861 non-null  float64
 5   length            15861 non-null  float64
 6   replacement_cost  15861 non-null  float64
 7   special_features  15861 non-null  object 
 8   NC-17             15861 non-null  int64  
 9   PG                15861 non-null  int64  
 10  PG-13             15861 non-null  int64  
 11  R                 15861 non-null  int64  
 12  amount_2          15861 non-null  float64
 13  length_2          15861 non-null  float64
 14  rental_rate_2     15861 non-null  float64
dtypes: float64(8), int64(4), object(3)
memory usage: 1.8+ MB


In [71]:
# rental_length_days

df_rental["rental_length"] = pd.to_datetime(df_rental["return_date"]) - pd.to_datetime(df_rental["rental_date"])
df_rental["rental_length_days"] = df_rental["rental_length"].dt.days

In [72]:
# dummy variables from 'special_features'
df_rental['deleted_scenes'] = np.where(df_rental['special_features'].str.contains("Deleted Scenes"), 1, 0)
df_rental['behind_the_scenes'] = np.where(df_rental['special_features'].str.contains("Behind the Scenes"), 1, 0)

In [73]:
df_rental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15861 entries, 0 to 15860
Data columns (total 19 columns):
 #   Column              Non-Null Count  Dtype          
---  ------              --------------  -----          
 0   rental_date         15861 non-null  object         
 1   return_date         15861 non-null  object         
 2   amount              15861 non-null  float64        
 3   release_year        15861 non-null  float64        
 4   rental_rate         15861 non-null  float64        
 5   length              15861 non-null  float64        
 6   replacement_cost    15861 non-null  float64        
 7   special_features    15861 non-null  object         
 8   NC-17               15861 non-null  int64          
 9   PG                  15861 non-null  int64          
 10  PG-13               15861 non-null  int64          
 11  R                   15861 non-null  int64          
 12  amount_2            15861 non-null  float64        
 13  length_2            15861 non-n

In [74]:
# X and y
columns_to_drop = ['rental_date','return_date','special_features','rental_length_days', 'rental_length']
X = df_rental.drop(columns_to_drop, axis=1)
y = df_rental['rental_length_days']

In [75]:
# Split into train and test
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                   test_size=0.2,
                                                   random_state=9)

In [76]:
from sklearn.linear_model import Lasso
# Create the Lasso model
lasso = Lasso(alpha=0.3, random_state=9) 

# Train the model and access the coefficients
lasso.fit(X_train, y_train)
lasso_coef = lasso.coef_

# Perform feature selectino by choosing columns with positive coefficients
X_lasso_train, X_lasso_test = X_train.iloc[:, lasso_coef > 0], X_test.iloc[:, lasso_coef > 0]

In [77]:
# Run OLS models on lasso chosen regression
from sklearn.linear_model import LinearRegression

ols = LinearRegression()
ols = ols.fit(X_lasso_train, y_train)
y_test_pred = ols.predict(X_lasso_test)
mse_lin_reg_lasso = mean_squared_error(y_test, y_test_pred)
rmse_lin = mse_lin_reg_lasso**(1/2)
rmse_lin

2.1936948833591794

In [78]:
# Decision Tree Regressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error as MSE

dt = DecisionTreeRegressor(max_depth=4,
                          min_samples_leaf=0.2,
                          random_state=SEED)

dt.fit(X_train, y_train)
y_pred = dt.predict(X_test)
mse_dt = MSE(y_test, y_pred)
rmse_test_dt = MSE(y_test, y_pred)**(1/2)
rmse_test_dt

2.1218841827374146

In [79]:
# Random Forest Regressor
from sklearn.ensemble import RandomForestRegressor

# set seed
SEED = 9

# instantiate model
rf = RandomForestRegressor(n_estimators=200,
                           min_samples_leaf=0.2,
                           random_state = SEED)

rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
rmse_test = MSE(y_test, y_pred)**(1/2)

rmse_test

2.382022612818934

In [80]:
best_mse = rmse_test
best_model = rf